In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pickle

# all-time top 3000 albums from acclaimed music
url_4 = 'https://acclaimedmusic.net/year/alltime_albums.htm'
# https://acclaimedmusic.net/year/alltime_albums_yearsort.htm

#  use requests to fetch the HTML content 
response = requests.get(url_4)

# Extract Data from HTML Files

# Create Beautiful Soup object passing HTML content, parser library (usually 'html.parser') as arguments.
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup.prettify())


In [13]:
#Method ONE to pull the HTML table data into a Dataframe:
table = soup.find('table', {'class': 'center'})

# Use pandas to read the HTML table
df = pd.read_html(str(table), header=0)[0]

# Display the DataFrame
display(df.head())
df.columns
df.rename(columns={'Rank▼': 'Acclaimed Rank', 
                   'Artist▼': 'Artist', 'Album▼': 'Album'  , 
                   'Genres▼': 'Genres', 'Year▼': 'Year'}, inplace=True)
display(df.head(10))
display(df.info())

C:\Users\black\AppData\Local\Temp\ipykernel_6552\2647961702.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=0)[0]


,Rank▼,Artist▼,Album▼,Genres▼,Year▼
0,1,The Beach Boys,Pet Sounds,Baroque Pop,1966
1,2,Nirvana,Nevermind,Grunge,1991
2,3,The Beatles,Revolver,Pop Rock,1966
3,4,The Velvet Underground & Nico,The Velvet Underground & Nico,"Art Rock, Experimental Rock",1967
4,5,Marvin Gaye,What's Going On,Soul,1971


,Acclaimed Rank,Artist,Album,Genres,Year
0,1,The Beach Boys,Pet Sounds,Baroque Pop,1966
1,2,Nirvana,Nevermind,Grunge,1991
2,3,The Beatles,Revolver,Pop Rock,1966
3,4,The Velvet Underground & Nico,The Velvet Underground & Nico,"Art Rock, Experimental Rock",1967
4,5,Marvin Gaye,What's Going On,Soul,1971
5,6,The Beatles,Sgt. Pepper's Lonely Hearts Club Band,"Pop Rock, Psychedelic Pop",1967
6,7,The Clash,London Calling,Punk Rock,1979
7,8,Radiohead,OK Computer,"Alternative Rock, Art Rock",1997
8,9,Bob Dylan,Blonde on Blonde,"Folk Rock, Singer/Songwriter",1966
9,10,The Rolling Stones,Exile on Main St.,Blues Rock,1972


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Acclaimed Rank  3000 non-null   int64 
 1   Artist          3000 non-null   object
 2   Album           3000 non-null   object
 3   Genres          3000 non-null   object
 4   Year            3000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 117.3+ KB


None

In [3]:
# # Method TWO to pull the HTML table data into a Dataframe:
# # Use Pandas to read the HTML content
# tables = pd.read_html(html_content)

# # Print the number of tables found
# print(f"Number of tables found: {len(tables)}")

# # Select the first table (assuming it's the one we want)
# df = tables[0]

# # Print the DataFrame
# print(df)

In [15]:
# Extract values for all the href values
# https://acclaimedmusic.net/album/A62.htm

href_a_elements = soup.find_all('a')

# Extract the values of the href attributes
hrefs = [a['href'] for a in href_a_elements]
df_hrefs = pd.DataFrame(hrefs, columns=['link'])

display(df_hrefs.head(10))
display(df_hrefs.info())

,link
0,alltime_albums.htm
1,alltime_albums_artistsort.htm
2,alltime_albums_albumsort.htm
3,alltime_albums_genresort.htm
4,alltime_albums_yearsort.htm
5,../artist/The Beach Boys.htm
6,../album/A62.htm
7,../genre/genre48.htm
8,../genre/ .htm
9,../genre/ .htm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21005 entries, 0 to 21004
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    21005 non-null  object
dtypes: object(1)
memory usage: 164.2+ KB


None

In [24]:
# Extract the album links  # ../album/A62.htm
df_album_hrefs = df_hrefs[df_hrefs['link'].str.contains('../album')]    \
                            .reset_index(drop=True)   \
                            .rename(columns={'link':'album_link'})
display(df_album_hrefs.head())
df_album_hrefs.info()

,album_link
0,../album/A62.htm
1,../album/A811.htm
2,../album/A63.htm
3,../album/A91.htm
4,../album/A211.htm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   album_link  3000 non-null   object
dtypes: object(1)
memory usage: 23.6+ KB


In [25]:
# add the Album_Link dataframe to main dataframe
df_combined = pd.concat([df, df_album_hrefs], axis='columns')
display(df_combined.head())
df_combined.info()


,Acclaimed Rank,Artist,Album,Genres,Year,album_link
0,1,The Beach Boys,Pet Sounds,Baroque Pop,1966,../album/A62.htm
1,2,Nirvana,Nevermind,Grunge,1991,../album/A811.htm
2,3,The Beatles,Revolver,Pop Rock,1966,../album/A63.htm
3,4,The Velvet Underground & Nico,The Velvet Underground & Nico,"Art Rock, Experimental Rock",1967,../album/A91.htm
4,5,Marvin Gaye,What's Going On,Soul,1971,../album/A211.htm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Acclaimed Rank  3000 non-null   int64 
 1   Artist          3000 non-null   object
 2   Album           3000 non-null   object
 3   Genres          3000 non-null   object
 4   Year            3000 non-null   int64 
 5   album_link      3000 non-null   object
dtypes: int64(2), object(4)
memory usage: 140.8+ KB


In [20]:
df_combined = df_combined.convert_dtypes(str)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Acclaimed Rank  3000 non-null   Int64 
 1   Artist          3000 non-null   string
 2   Album           3000 non-null   string
 3   Genres          3000 non-null   string
 4   Year            3000 non-null   Int64 
 5   album_link      3000 non-null   string
dtypes: Int64(2), string(4)
memory usage: 146.6 KB


In [23]:
# Save as Pickle:
import pickle
with open('../Resources/acclaimed_albums_genres.pkl', 'wb') as file:
    # Use pickle.dump to store the list of images
    pickle.dump(df_combined, file)

# Reload Dataframe from pkl file, with rb 'read binary'
# with open('../Resources/acclaimed_albums_genres.pkl', 'rb') as file:
#     df = pickle.load(file)


In [22]:
#Export csv
df_combined.to_csv("../Resources/acclaimed_albums_genres.csv", index=False)

In [ ]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.

# The rest of this is to get the Allmusic Ratings...
#  MOVED this code to moved to acclaimed_amg_ratings.ipynb.


In [50]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.


# import warnings
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [52]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.
# get album names, ratings
counter = 0
artist_album_names = []
all_genres_list = []
ratings = []


for link in df_combined['album_link']:
    counter +=1
    print(link)
    response = requests.get(link, allow_redirects=False)
    soup = BeautifulSoup(response.text, 'html.parser')

 # Find the first 'strong' section to get Album Title and Band Name   
    for word in soup.find('strong'): 
        name=word.get_text(strip=True)
        if name != '':
            artist_album_names.append(name)

# Find the genres section
    genres_section = soup.find('div', class_='auto-style3')
    genre_links = genres_section.find_all('a', href=True)
# Extract and print the text content of each genre
    genres = [link.get_text() for link in genre_links if 'genre' in link['href']]
    if genres != '':
        while(" " in genres):
            genres.remove(" ")
        all_genres_list.append(genres)


# Find the specific section with the All Music Guide rating
    try:
        all_music_guide_section = soup.find('a', \
            href='http://www.allmusic.com/index.html').find_parent('td').find_next_sibling('td')
    except Exception as e:
        print(f'{e}; Did not find allmusic info for {link}')

    rating = all_music_guide_section.span.get_text(strip=True)
    ratings.append(rating)
    
    # if counter == 5:
    #     break

print(f'artist_album names {len(artist_album_names)}:  {artist_album_names}')
print(f'all genres {len(all_genres_list)}: {all_genres_list}')
print(f'ratings {len(ratings)}: {ratings}')

https://acclaimedmusic.net/album/A62.htm
<!DOCTYPE html>
<html>
 <head>
  <title>
   Acclaimed Music
  </title>
  <meta content="Acclaimed Music. The most recommended albums and songs of all time." name="description"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="Acclaimed music, Recommended albums, recommended songs, Best albums, Best songs, best of list, Critic lists, Critics lists, Music lists, Rock lists, music magazines" name="keywords"/>
  <link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="../styles.css" rel="stylesheet"/>
  <style>
   @media only screen and (min-device-width : 1201px){ .home, .mobile { display: none;}}
@media only screen and (min-device-width : 0px) and (max-device-width : 1200px){.main-item1, .notmobile, .imgnotmobile, .imgnotmobile2 { display: none;}}
@media only screen and (min-device-width : 0px) and (max-device-width : 1200px){.main-item2alp a:link,.main-item2alp a:vi

C:\Users\black\AppData\Local\Temp\ipykernel_14612\549344249.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  genres_section = soup.find(text="Genres:")


<!DOCTYPE html>
<html>
 <head>
  <title>
   Acclaimed Music
  </title>
  <meta content="Acclaimed Music. The most recommended albums and songs of all time." name="description"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="Acclaimed music, Recommended albums, recommended songs, Best albums, Best songs, best of list, Critic lists, Critics lists, Music lists, Rock lists, music magazines" name="keywords"/>
  <link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="../styles.css" rel="stylesheet"/>
  <style>
   @media only screen and (min-device-width : 1201px){ .home, .mobile { display: none;}}
@media only screen and (min-device-width : 0px) and (max-device-width : 1200px){.main-item1, .notmobile, .imgnotmobile, .imgnotmobile2 { display: none;}}
@media only screen and (min-device-width : 0px) and (max-device-width : 1200px){.main-item2alp a:link,.main-item2alp a:visited,.main-item2alp a:active {font-weigh

In [67]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.

# Break out Band name, Album, and Rating into Diff Lists based on odd/even Index
# Can't use index, since some items have the same album/artist name, and it will
#  get the first one.
 #   ind = artist_album_names.index(item)

artist_names = []
album_names = []

counter = 0
for item in artist_album_names:
    print(counter, item)
    if counter % 2 == 0: # 0, even
        artist_names.append(item)
        counter +=1
    else:
        album_names.append(item)
        counter +=1

print('artists', artist_names)
print('albums', album_names)

0 The Beach Boys
1 Pet Sounds
2 Nirvana
3 Nevermind
4 The Beatles
5 Revolver
6 The Velvet Underground & Nico
7 The Velvet Underground & Nico
8 Marvin Gaye
9 What's Going On
10 The Beatles
11 Sgt. Pepper's Lonely Hearts Club Band
12 The Clash
13 London Calling
14 Radiohead
15 OK Computer
16 Bob Dylan
17 Blonde on Blonde
18 The Rolling Stones
19 Exile on Main St.
20 Bob Dylan
21 Highway 61 Revisited
22 Sex Pistols
23 Never Mind the Bollocks - Here's the Sex Pistols
24 The Beatles
25 The Beatles ["White Album"]
26 The Jimi Hendrix Experience
27 Are You Experienced?
28 Van Morrison
29 Astral Weeks
30 Bruce Springsteen
31 Born to Run
32 Kendrick Lamar
33 To Pimp a Butterfly
34 Public Enemy
35 It Takes a Nation of Millions to Hold Us Back
36 David Bowie
37 The Rise and Fall of Ziggy Stardust and the Spiders from Mars
38 The Beatles
39 Abbey Road
40 Pink Floyd
41 The Dark Side of the Moon
42 Bob Dylan
43 Blood on the Tracks
44 Michael Jackson
45 Thriller
46 Arcade Fire
47 Funeral
48 Patti Smi

In [68]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.

# Add artists, albums, and ratings (all shold be 5 star) to a new Dataframe
dic = {'acc_artist'  : artist_names, 
       'acc_album'   : album_names, 
       'all genres'  : genres_list,
       'amg_rating'  : ratings}

allmusic_df = pd.DataFrame(dic)
allmusic_df.head()

,acc_artist,acc_album,genres,amg_rating
0,The Beach Boys,Pet Sounds,"[Baroque Pop, Pop]",5 Stars
1,Nirvana,Nevermind,"[Grunge, Alternative Rock, Rock]",5 Stars
2,The Beatles,Revolver,"[Pop Rock, Pop, Rock]",5 Stars
3,The Velvet Underground & Nico,The Velvet Underground & Nico,"[Art Rock, Experimental Rock, Rock]",5 Stars
4,Marvin Gaye,What's Going On,"[Soul, R&B]",5 Stars


In [70]:
#combine data frames
df_super = pd.concat([allmusic_df, df_combined], axis=1)
df_super.head()
df_super.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   acc_artist  3000 non-null   object
 1   acc_album   3000 non-null   object
 2   genres      3000 non-null   object
 3   amg_rating  3000 non-null   object
 4   album_link  3000 non-null   string
 5   artist      3000 non-null   string
 6   year        3000 non-null   string
dtypes: object(4), string(3)
memory usage: 164.2+ KB


In [71]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.

import csv
df_super.to_csv("allmusic_5star_super.csv", index=False)

In [ ]:
# <a href="../artist/The Beach Boys.htm">  The Beach Boys
# <a href="../album/A62.htm">              Pet Sounds
# <a href="../genre/genre48.htm">          Baroque Pop
# <a href="../year/1966a.htm">             1966

# <tr>
#     <td style="text-align:center">
#         <div class="font12">
#             1
#         </div>
#     </td>
#     <td>
#         <a href="../artist/The Beach Boys.htm">
#             The Beach Boys
#         </a>
#     </td>
#     <td>
#         <a href="../album/A62.htm">
#             Pet Sounds
#         </a>
#     </td>
#     <td class="notmobile">
#         <a href="../genre/genre48.htm">
#             Baroque Pop
#         </a>
#         <!-- other genre links -->
#     </td>
#     <td class="notmobile" style="text-align:center">
#         <a href="../year/1966a.htm">
#             1966
#         </a>
#     </td>
# </tr>

In [ ]:
#  NEXT TO DO:

# + link to the AllMusic page with the review?

# Pet Sounds
# https://acclaimedmusic.net/album/A62.htm
 
# Links of interest:
# https://rateyourmusic.com/release/album/the-beach-boys/pet-sounds/

# at all music:
# https://www.allmusic.com/search/all/beach%20boys%2C%20pet%20sounds
# https://www.allmusic.com/album/pet-sounds-mw0000398074

# at Amazon
# https://www.amazon.com/s?k=music%3A+%22Pet+Sounds%22+by+%22The+Beach+Boys
# https://www.amazon.com/s?k=music%3A+%22Pet+Sounds%22+by+%22The+Beach+Boys%22+vinyl


In [ ]:
#  **NOT USED** : moved to acclaimed_amg_ratings.ipynb.

##########################################
# SANDBOX: get all the Rank Sources and their Ranks in a DataFrame
# Fetch response/soup for ONE Album Link

url = 'https://acclaimedmusic.net/album/A62.htm'
#  use requests to fetch the HTML content 
response = requests.get(url, allow_redirects=False)
# Create Beautiful Soup object passing HTML content, parser library: 'html.parser' as arg
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup.prettify())

################################################
#  create a dateframe of all the Table entites.
df = pd.read_html(str(soup), displayed_only=False)[0]
df_ranks = df.rename(columns={0: 'source', 1 : 'rank' })
display(df_ranks.head())
display(df.info())

,source,rank
0,"Best of All-time Lists, Best of Decade Lists, ...",NaN
1,"Allan Kozinn, AVGuide (USA) - The 100 Greatest...",No Order
2,Amazon.com (USA) - The 10 Best Albums by Decad...,7
3,Bill Shapiro (USA) - The Top 100 Rock Compact ...,No Order
4,Blender (USA) - 500 CDs You Must Own Before Yo...,No Order


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       179 non-null    object
 1   1       177 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


None